## 使用离散余弦变换（DCT）压缩语音信号。

首先载入库函数和依赖包。

In [1]:
using MAT
using TyPlot
using TySignalProcessing
using TyMath
using TyBase
using Printf


加载一个包含单词“strong”的文件，由一个女人和一个男人说出。信号以8 kHz采样。

In [2]:

matfile = matread("strong.mat")

Dict{String, Any} with 15 entries:
  "ypc"  => 57.3928
  "x"    => [-0.00109142 -0.00588304 … -0.00344999 -0.00180392]
  "XX"   => [0.590874 0.574983 … 3.90454e-6 2.40094e-6]
  "need" => 2395.0
  "y"    => [0.00384905 -0.0571342 … -0.0109486 0.00168894]
  "fs"   => 8000.0
  "X"    => [0.0 0.0 … 0.0 0.0]
  "xx"   => [0.00323139 0.00164978 … -0.00343413 -0.00443751]
  "her"  => [-0.00109142; -0.00588304; … ; -0.00344999; -0.00180392;;]
  "Y"    => [0.0 0.0 … 0.0 0.0]
  "ind"  => [1030.0 1032.0 … 2135.0 4136.0]
  "YY"   => [1.39744 1.32757 … 3.33857e-5 2.27489e-5]
  "him"  => [0.00384905; -0.0571342; … ; -0.0109486; 0.00168894;;]
  "yy"   => [0.00554873 -0.0531845 … -0.00521347 0.00253385]
  "xpc"  => 44.9479

利用离散余弦变换对女声信号进行压缩。将信号分解为DCT基向量。分解中的项与信号中的样本一样多。矢量X中的展开系数测量在每个分量中存储了多少能量。将系数从大到小排序。

In [3]:
# 女声分析
her = matfile["her"]
x = her;
X = dct(x);
XX = sort(abs.(X), lt=!isless, dims=1)
X = vec(X)
indx = sortperm(abs.(vec(X)), lt=!isless)

4899-element Vector{Int64}:
  269
  299
  260
  258
  306
  307
  298
  305
  518
  304
    ⋮
 4285
 4429
 4859
 4765
 4737
 3907
 2735
 2186
 2665

找出多少DCT系数代表信号中99.9%的能量。把这个数字表示为总数的百分比。将包含剩余0.1%能量的系数设置为零。从压缩表示重建信号。绘制原始信号、其重建以及两者之间的差异。

In [4]:

needx = 1;
while norm(X[indx[1:needx]]) / norm(X) < 0.999
    global needx = needx + 1
end

xpc = needx / length(X) * 100;

X[indx[needx+1:end]] .= 0;
xx = idct(X);

figure(1)
TyPlot.plot(x)
hold("on")
TyPlot.plot(xx)
hold("on")
TyPlot.plot(x - xx)
coeffs_percentage = @sprintf("%d %s", xpc, "% of coeffs.")
legend(["Original", coeffs_percentage, "Difference"])

PyObject <objects.mw_legend.CLegend object at 0x0000021E7F590B48>

对**男性声音**重复分析。找出有多少DCT系数代表99.9%的能量，并将该数字表示为总数的百分比。

In [5]:
# 男声分析
him = matfile["him"]
y = him;
Y = dct(y);
YY = sort(abs.(Y), lt=!isless, dims=1)
Y = vec(Y)
indy = sortperm(abs.(vec(Y)), lt=!isless)

needy = 1;
while norm(Y[indy[1:needy]]) / norm(Y) < 0.999
    global needy = needy + 1
end

ypc = needy / length(Y) * 100;


将其余系数设置为零，并从压缩版本重建信号。绘制原始信号、其重建以及两者之间的差异。

In [6]:

Y[indy[needy+1:end]] .= 0;
yy = idct(Y);

figure(2)
TyPlot.plot(y)
hold("on")
TyPlot.plot(yy)
hold("on")
TyPlot.plot(y - yy)
coeffs_percentage = @sprintf("%d %s", ypc, "% of coeffs.")
legend(["Original", coeffs_percentage, "Difference"])

PyObject <objects.mw_legend.CLegend object at 0x0000021E12FA5308>

在这两种情况下，大约一半的DCT系数足以合理地重构语音信号。如果所需的能量分数为99%，则所需系数的数量减少到总数的约20%。由此产生的重建是低劣的，但仍然可以理解。对这些和其他样本的分析表明，需要更多的系数来表征男人的声音，而不是女人的声音。